In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# move into project directory
repo_name = "UMass-CS670-Project"
%cd /content/drive/MyDrive/UMass-Courses/Semester-2/CS670/$repo_name
!ls

/content/drive/MyDrive/UMass-Courses/Semester-2/CS670/UMass-CS670-Project
common			  get_dataset.sh	   models		  source-zip
datautils		  github-repo-clone.ipynb  preprocess_dataset.py  texture_params.txt
demo			  Index.ipynb		   README.md
excluded-train-files.txt  index.py		   SINet.ipynb
experiments		  last_run_info.json	   source-data


In [ ]:
# set up environment
# comment out if not required
'''
!pip3 install torch torchvision torchaudio
!pip install matplotlib numpy pandas pyyaml opencv-python
'''

'\n!pip3 install torch torchvision torchaudio\n!pip install matplotlib numpy pandas pyyaml opencv-python\n'

In [ ]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder
# uncomment the lines below  to download the data

import os

!pip install gdown

data_dir="source-data"

if not(os.path.exists("source-data")):
    !gdown https://drive.google.com/uc?id=18oBjWeuw5qAq4HG_jZdjUHas4APy-KJE&export=download
    #echo 'Downloaded data! Unzipping to data folder'
    !unzip -qq -d . ./COD10K-v3.zip
    os.rename("./COD10K-v3.zip", "./source-data/COD10K-v3.zip")
else:
    print("\nThe data directory exists!")

In [4]:
# setup some imports
from datautils.datareader import read_data
from datautils.dataset import COD10KDataset
from torch.utils.data import DataLoader
import random
import numpy as np
import torch
import argparse
from tqdm import tqdm
from torchvision.io import read_image
from torchvision.utils import save_image
from torchvision.transforms.functional import to_pil_image
import json

from experiments.style_transfer import style_transfer
from common.visualizer import layer_visualizer

In [5]:
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [6]:
def run_style_transfer_pipeline(args, texture_name, style_weight, last_batch_run = -1):
    pos_data_paths = read_data('Train')

    dataset = COD10KDataset(pos_data_paths)
    dataloader = DataLoader(dataset, batch_size = args.batch_size, shuffle = False)

    for i_batch, batch in enumerate(dataloader):
        if i_batch > last_batch_run:
            print(f"Processing batch {i_batch}, image: {batch['img_name']} of dimensions: {batch['img'].shape}")
            style_img = read_image(f'./source-data/Textures/{texture_name}.jpg')
            new_img = style_transfer(
                batch['img'],
                style_img,
                [0, 2, 5, 14, 23],
                21,
                1e-4,
                [style_weight]*5,
                1e-5,
                args)
            img_name = batch['img_name'][0]
            img_name = img_name.replace(".jpg", "")
            img_name = f"./source-data/Train/Styled-Image/{img_name}-Texture-{texture_name}.jpg"
            save_image(new_img, img_name)
            last_run = {
                "last_batch": i_batch
            }
            print(f"completing transfer of img {i_batch} with texture {texture_name}")
            with open("./last_run_info.json", "w") as fp:
                json.dump(last_run, fp)

    print(f'Finished modifying train dataset images for {texture_name}')
    last_run = {
        "last_batch": -1
    }
    with open("./last_run_info.json", "w") as fp:
        json.dump(last_run, fp)
    '''
    style_img = read_image(f'./data/Textures/{texture_name}.jpg')
    content_img = read_image('./data/Mini-Set/butterfly-image.jpg')
    #content_img = batch['img']
    print('b4 trans', content_img.shape)
    new_img = style_transfer(content_img,
            style_img,
            [0, 2, 5, 14, 23],
            21,
            1e-4,
            [style_weight]*5,
            1e-5,
            args)
    #pi = to_pil_image(pi)
    #plt.axis("off")
    #plt.imshow(pi)
    #plt.show()
    save_image(new_img, f'./data/COD10K-v3/Train/Styled-Image/butterfly-image-Texture-{texture_name}.jpg')
    #print('after trans', new_img.size())
    '''
    return -1

In [ ]:

param_dict = {
    'moss': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'snow': {
        'epochs': 200,
        'style_weights': 1.5
    },
    'wet-sand': {
        'epochs': 200,
        'style_weights': 3.5
    },
    'sand-with-moss': {
        'epochs': 100,
        'style_weights': 2.5
    },
    'rain': {
        'epochs': 100,
        'style_weights': 4.5
    },
    'wood': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'grass': {
        'epochs': 100,
        'style_weights': 0.1
    },
    'foliage-texture': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'blue-coral': {
        'epochs': 100,
        'style_weights': 1.1
    }
}

with open("./last_run_info.json") as fp:
    last_run = json.load(fp)["last_batch"]

selected_textures = ['moss', 'wet-sand', 'sand-with-moss', 'rain', 'foliage-texture']

for texture in selected_textures:

    parser = argparse.ArgumentParser()
    parser.add_argument('--batch_size', type = int, default = 1)
    parser.add_argument('--model_name', type=str, default='vgg')
    parser.add_argument('--lr', type=float, default=0.05)
    parser.add_argument('--max_iter', type=int, default=param_dict[texture]['epochs'])
    args = parser.parse_args(args=[])

    #style_img = read_image(f'./data/Textures/{texture}.jpg')
    #print(img.size())
    #layer_visualizer(img, args)

    run_style_transfer_pipeline(args, texture, param_dict[texture]['style_weights'], last_run)

In [ ]:
from datautils.datareader import read_data
from torch.utils.data import DataLoader
from datautils.dataset import COD10KDataset
from models.custom_models import get_model
from torchvision.transforms import transforms
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F

def image_collate(batch):
    batchlist = list(map(list, zip(*batch)))
    return batchlist

def __convert_to_grascale(img):
    imin, imax = img.min(), img.max()
    x = (img - imin) / (imax - imin)
    return x

def __prepare_data(type = 'Train'):
    data_paths = read_data('Train', True)
    dataset = COD10KDataset(data_paths, 'binary_classification')
    return dataset

def binary_classification(args):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((args.resize_size, args.resize_size)),
        transforms.Lambda(__convert_to_grascale),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])
    ])
    train_dataset = __prepare_data()
    train_dataloader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle = False, collate_fn = image_collate)
    model = get_model(2, "vit")

    optimizer = torch.optim.Adam(model.parameters(), lr = args.lr, betas = (0.99, 0.999))
    loss_fn = torch.nn.BCELoss()
    train_losses = []

    for _ in range(args.max_iter):
        avg_loss = 0
        for i_batch, batch in enumerate(train_dataloader):
            #print(batch[1])
            lbls = torch.tensor(batch[1])
            #inds = torch.tensor(list(range(len(batch[1]))))
            print(lbls.size(), lbls == 0)
            target = torch.zeros((len(batch[1]), 2))
            target[lbls == 1, 1] = 1.0
            target[lbls == 0, 0] = 0.0
            #print(target.size())
            #print(target)
            optimizer.zero_grad()
            #print(len(batch), type(batch[0]), len(batch[0]))
            img_batch = list(map(transform, batch[0]))
            img_batch = torch.stack(img_batch, 0)
            #print(img_batch.size())
            #tr_batch = transform(batch['img'])
            op = F.softmax(model(img_batch))
            loss = loss_fn(op, target)
            loss.backward()
            avg_loss += loss.item()
            #print(op.size())
            #break
        train_losses.append(avg_loss / (i_batch + 1))
    plt.plot(list(range(args.max_iter)), train_losses)
    plt.title('Loss - Epoch plot')
    plt.show()

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type = int, default = 16)
parser.add_argument('--model_name', type=str, default='vit')
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--max_iter', type=int, default=1)
parser.add_argument('--resize_size', type=int, default=224)
args = parser.parse_args(args=[])

binary_classification(args)

torch.Size([16]) tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])


<ipython-input-10-e6f26cb4c29d>:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  op = F.softmax(model(img_batch))


torch.Size([16]) tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])
torch.Size([16]) tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])
torch.Size([16]) tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])
torch.Size([16]) tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])
torch.Size([16]) tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])
torch.Size([16]) tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])
torch.Size([16]) tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, 